In [1]:
import pandas as pd

import numpy as np

import csv,sqlite3

In [2]:
conn = sqlite3.connect("predictions_sql.db")
cursor = conn.cursor()

In [3]:
df = pd.read_csv("predictions.csv", header=None, names=["user_id", "item_id", "rating_5"])

In [4]:
# df.to_sql("predictions_tb", conn, index=False)

In [6]:
# read from sql db
# pred_df = pd.read_sql_query("select * from predictions_tb limit 5;", conn)

df.head()

,user_id,item_id,rating_5
0,A2DJV5P2NTWEK0,0001981692,4.191430
1,A2DJV5P2NTWEK0,0002008211,4.699676
2,A2DJV5P2NTWEK0,0002254565,4.702403
3,A2DJV5P2NTWEK0,0002552493,4.645525
4,A2DJV5P2NTWEK0,0002710188,4.791867


In [7]:
# store pickle files
metadata = pd.read_pickle("refined_metadata.pkl")

metadata1 = metadata[['asin', 'title', 'imUrl', 'authors', 'lexile']]

metadata1['lexile'] = pd.to_numeric(np.where(metadata1['lexile'].str[0].str.isalpha()\
                                             , metadata1['lexile'].str[2:-1], metadata1['lexile'].str[:-1]))

metadata1.head(10)

/home/apoorv/anaconda3/envs/cs670/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,asin,title,imUrl,authors,lexile
0,0001006657,I Want My Dinner,http://ecx.images-amazon.com/images/I/510AQCZV...,"[{""name"":""Tony Ross"",""url"":""/content/scholasti...",240
1,0001981692,Pirates,http://ecx.images-amazon.com/images/I/51WN7BJM...,"[{""name"":""Jim Brew"",""url"":""/content/scholastic...",770
2,0002008211,Haunted,http://ecx.images-amazon.com/images/I/516OPgTS...,"[{""name"":""R. L. Stine"",""url"":""/content/scholas...",460
3,0002199203,Rhino,http://ecx.images-amazon.com/images/I/5189PrGP...,"[{""name"":""Sheila Solomon Klass"",""url"":""/conten...",660
4,0002254565,Footsteps,http://ecx.images-amazon.com/images/I/51NVFMSP...,"[{""name"":""Pamela Rushby"",""url"":""/content/schol...",260
5,0002552493,Hollywood Dogs,http://ecx.images-amazon.com/images/I/41i7XNGU...,"[{""name"":""Meish Goldish"",""url"":""/content/schol...",720
6,0002710188,Home Run,http://ecx.images-amazon.com/images/I/91DoEE3K...,"[{""name"":""Sara Hoskinson Frommer"",""url"":""/cont...",180
7,0003002241,Stuart Little,NaN,"[{""name"":""Garth Williams"",""url"":""/content/scho...",500
8,0006140882,Ransom,NaN,"[{""name"":""Saddleback Publishing"",""url"":""/conte...",340
9,0006137989,Crooked House,http://ecx.images-amazon.com/images/I/51lea3n2...,"[{""name"":""Agatha Christie"",""url"":""/content/sch...",630


In [8]:
# cursor.execute('''DROP TABLE IF EXISTS metadata_tbl''')
# conn.commit()

# metadata1.to_sql("metadata_tbl", conn, index=False)

In [9]:
# store utility file
utility = pd.read_pickle("utility_df.pkl")

In [10]:
utility1 = utility.stack().to_frame("rating_5")

utility1.reset_index(level=1, inplace=True)
utility1.reset_index(level=0, inplace=True)

utility1.head(10)

,user,item,rating_5
0,A03208803KA5HCUYS17Z1,0062117378,5.0
1,A03208803KA5HCUYS17Z1,0062235656,5.0
2,A03208803KA5HCUYS17Z1,014241543X,5.0
3,A03208803KA5HCUYS17Z1,0142419400,4.0
4,A03208803KA5HCUYS17Z1,0374351384,3.0
5,A03208803KA5HCUYS17Z1,0375866566,2.0
6,A100YDQ9DIP06P,0060589507,5.0
7,A100YDQ9DIP06P,0061130990,3.0
8,A100YDQ9DIP06P,0061730831,5.0
9,A100YDQ9DIP06P,0061921033,5.0


In [11]:
# cursor.execute('''DROP TABLE IF EXISTS utility_tbl''')
# conn.commit()

# utility1.to_sql("utility_tbl", conn, index=False)

In [12]:
# get user lexile rating
metadata2 = metadata1[["asin","lexile"]]

In [13]:
usr_lxl = utility1.merge(metadata2, how='left', left_on='item', right_on='asin')
usr_lxl.head(10)

,user,item,rating_5,asin,lexile
0,A03208803KA5HCUYS17Z1,0062117378,5.0,0062117378,610
1,A03208803KA5HCUYS17Z1,0062235656,5.0,0062235656,580
2,A03208803KA5HCUYS17Z1,014241543X,5.0,014241543X,830
3,A03208803KA5HCUYS17Z1,0142419400,4.0,0142419400,580
4,A03208803KA5HCUYS17Z1,0374351384,3.0,0374351384,720
5,A03208803KA5HCUYS17Z1,0375866566,2.0,0375866566,760
6,A100YDQ9DIP06P,0060589507,5.0,0060589507,150
7,A100YDQ9DIP06P,0061130990,3.0,0061130990,760
8,A100YDQ9DIP06P,0061730831,5.0,0061730831,490
9,A100YDQ9DIP06P,0061921033,5.0,0061921033,720


In [14]:
# We will require timestamp, otherwise hard to defend.
# sort usr_lxl at user, rating level & select first row for each user
usr_lxl2 = usr_lxl.sort_values(by=['user', 'rating_5'], ascending=False).groupby('user').first().reset_index()
usr_lxl3 = usr_lxl2[["user","lexile"]].rename(columns={'lexile':'lexile_user'})

usr_lxl3.head()

,user,lexile_user
0,A03208803KA5HCUYS17Z1,610
1,A100YDQ9DIP06P,150
2,A100ZQDV7L8PVV,1250
3,A104BOQOO5T2SU,430
4,A104SG9QZ92UAH,620


In [15]:
# merge user lexile ratings with predictions data & join lexile ratings of books
df2 = df.merge(usr_lxl3, how='left', left_on='user_id', right_on='user').drop('user',axis=1)\
        .merge(metadata1, how='left', left_on='item_id', right_on='asin')

df2.head()

,user_id,item_id,rating_5,lexile_user,asin,title,imUrl,authors,lexile
0,A2DJV5P2NTWEK0,0001981692,4.191430,240,0001981692,Pirates,http://ecx.images-amazon.com/images/I/51WN7BJM...,"[{""name"":""Jim Brew"",""url"":""/content/scholastic...",770
1,A2DJV5P2NTWEK0,0002008211,4.699676,240,0002008211,Haunted,http://ecx.images-amazon.com/images/I/516OPgTS...,"[{""name"":""R. L. Stine"",""url"":""/content/scholas...",460
2,A2DJV5P2NTWEK0,0002254565,4.702403,240,0002254565,Footsteps,http://ecx.images-amazon.com/images/I/51NVFMSP...,"[{""name"":""Pamela Rushby"",""url"":""/content/schol...",260
3,A2DJV5P2NTWEK0,0002552493,4.645525,240,0002552493,Hollywood Dogs,http://ecx.images-amazon.com/images/I/41i7XNGU...,"[{""name"":""Meish Goldish"",""url"":""/content/schol...",720
4,A2DJV5P2NTWEK0,0002710188,4.791867,240,0002710188,Home Run,http://ecx.images-amazon.com/images/I/91DoEE3K...,"[{""name"":""Sara Hoskinson Frommer"",""url"":""/cont...",180


In [16]:
usr_itm_lxl = df2
usr_itm_lxl["lexile_diff"] = abs(usr_itm_lxl["lexile_user"] - usr_itm_lxl["lexile"])

In [17]:
usr_itm_lxl2 = usr_itm_lxl.drop(['asin'],axis=1).sort_values(by=['user_id', 'lexile_diff'], ascending=[False, True])

In [18]:
usr_itm_lxl3 = usr_itm_lxl2.groupby('user_id').head(200)

In [19]:
usr_itm_lxl3.head()

,user_id,item_id,rating_5,lexile_user,title,imUrl,authors,lexile,lexile_diff
2389317,AZXNB4Q5QCO2P,0060193263,4.058366,1050,Cowboy,http://ecx.images-amazon.com/images/I/51X1ZSJ4...,"[{""name"":""Robert Klausmeier"",""url"":""/content/s...",1050,0
2389627,AZXNB4Q5QCO2P,0061240273,3.918141,1050,Belong to Me,http://ecx.images-amazon.com/images/I/51iLPqq7...,"[{""name"":""Marisa De los Santos"",""url"":""/conten...",1050,0
2390561,AZXNB4Q5QCO2P,030726579X,3.994089,1050,Heart and Soul,http://ecx.images-amazon.com/images/I/71dHTFmJ...,"[{""name"":""Kadir Nelson"",""url"":""/content/schola...",1050,0
2390691,AZXNB4Q5QCO2P,031259612X,3.929125,1050,Wereling,http://ecx.images-amazon.com/images/I/41oRag1B...,"[{""name"":""Steve Feasey"",""url"":""/content/schola...",1050,0
2390741,AZXNB4Q5QCO2P,0316005363,4.074077,1050,Matter,http://ecx.images-amazon.com/images/I/41TE5qmy...,"[{""name"":""Christopher Cooper"",""url"":""/content/...",1050,0


In [20]:
cursor.execute('''DROP TABLE IF EXISTS usr_itm_lxl3_tbl''')
conn.commit()

usr_itm_lxl2.to_sql("usr_itm_lxl3_tbl", conn, index=False)

In [21]:
def filters(result, lxl_limit, rat_limit):
    if len(result)<10:
        cond = (usr_rows.lexile_diff <= lxl_limit) & (usr_rows.rating_5 >= rat_limit)
        rows = usr_rows.loc[cond,:]
        rows = rows.sort_values(by='rating_5', ascending=False)
        result = result.append(rows, ignore_index=True)
        usr_rows.drop(rows.index, inplace=True)
    return result

In [22]:
t_user = "AZXNB4Q5QCO2P"

In [23]:
usr_rows = usr_itm_lxl3[usr_itm_lxl3['user_id']==t_user]

result = pd.DataFrame(columns = usr_rows.columns)

result = filters(result,50,4.5)     # reco books with lexile_diff < 50 and rating >= 4.5
result = filters(result,50,4.0)
result = filters(result,100,4.5)
result = filters(result,100,4.0)

result = result.head(10)

/home/apoorv/anaconda3/envs/cs670/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [24]:
result

,user_id,item_id,rating_5,lexile_user,title,imUrl,authors,lexile,lexile_diff
0,AZXNB4Q5QCO2P,0142413836,4.553568,1050,Going Solo,http://ecx.images-amazon.com/images/I/710XJ1ji...,"[{""name"":""Roald Dahl"",""url"":""/content/scholast...",1080,30
1,AZXNB4Q5QCO2P,014039012X,4.495015,1050,"Narrative of the Life of Frederick Douglass, A...",http://ecx.images-amazon.com/images/I/51jk4fhU...,"[{""name"":""Peter J. Gomes"",""url"":""/content/scho...",1080,30
2,AZXNB4Q5QCO2P,0006371949,4.476489,1050,The Holocaust,http://ecx.images-amazon.com/images/I/41%2BOCl...,"[{""name"":""Philip Steele"",""url"":""/content/schol...",1080,30
3,AZXNB4Q5QCO2P,0375829709,4.459889,1050,Babe: The Gallant Pig,http://ecx.images-amazon.com/images/I/518XK75D...,"[{""name"":""Mary Rayner"",""url"":""/content/scholas...",1040,10
4,AZXNB4Q5QCO2P,014241381X,4.458914,1050,Boy,http://ecx.images-amazon.com/images/I/51F2xRzh...,"[{""name"":""Roald Dahl"",""url"":""/content/scholast...",1020,30
5,AZXNB4Q5QCO2P,0714845299,4.438892,1050,Nicholas,http://ecx.images-amazon.com/images/I/41JJD1JG...,"[{""name"":""Jean-Jacques Sempé"",""url"":""/content/...",1070,20
6,AZXNB4Q5QCO2P,0395329205,4.421599,1050,Castle,http://ecx.images-amazon.com/images/I/5138GYX1...,"[{""name"":""Mark Bergin"",""url"":""/content/scholas...",1080,30
7,AZXNB4Q5QCO2P,0547512910,4.414598,1050,Actual Size,http://ecx.images-amazon.com/images/I/51qObTY8...,"[{""name"":""Steve Jenkins"",""url"":""/content/schol...",1080,30
8,AZXNB4Q5QCO2P,0439042968,4.411782,1050,882 1/2 Amazing Answers to Your Questions Abou...,http://ecx.images-amazon.com/images/I/51bxzx4R...,"[{""name"":""Ken Marschall"",""url"":""/content/schol...",1050,0
9,AZXNB4Q5QCO2P,0451134699,4.407586,1050,2001: A Space Odyssey,http://ecx.images-amazon.com/images/I/51YGHJg1...,"[{""name"":""Arthur C. Clarke"",""url"":""/content/sc...",1060,10
